In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, wikipedia_page, to handle the contents of the website
wikipedia_page=requests.get(wikipedia_link)

#Store the contents of the website under wikipedia_page
wikipedia_page = lh.fromstring(wikipedia_page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = wikipedia_page.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [3]:
tr_elements = wikipedia_page.xpath('//tr')

#Create empty list
col=[]

i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(name)
    col.append((name,[]))

Postal Code

Borough

Neighborhood



In [4]:
col

[('Postal Code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[362, 362, 362]

In [8]:
# Transforming the dictionary on a dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [9]:
# Cleaning break lines
df = df.replace(r'\n','', regex=True)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#Removing Not assigned zip codes and fixing df index
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.index = range(len(df))
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
203,M4Y,Downtown Toronto,Church and Wellesley
204,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
205,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
206,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [11]:
df.shape

(208, 3)

In [29]:
#!conda install -c conda-forge geocoder --yes
import geopy
import geocoder
import pandas as pd
import numpy as np
import googlemaps

In [30]:
# initialize your variable to None
lat_lng_coords = None

#Create extra columns
df['Latitude'] = pd.Series("", index=df.index)
df['Longitude'] = pd.Series("", index=df.index)

df.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [39]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df3 = pd.merge(df,lat_lon,on='Postcode')
df3.head()

,Postcode,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,,,43.753259,-79.329656
1,M3A,North York,Parkwoods,,,43.753259,-79.329656
2,M4A,North York,Victoria Village,,,43.725882,-79.315572
3,M4A,North York,Victoria Village,,,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",,,43.654260,-79.360636


In [42]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,Postcode,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",,,43.654260,-79.360636
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",,,43.654260,-79.360636
8,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",,,43.662301,-79.389494
9,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",,,43.662301,-79.389494
18,M5B,Downtown Toronto,"Garden District, Ryerson",,,43.657162,-79.378937
...,...,...,...,...,...,...,...
195,M5X,Downtown Toronto,"First Canadian Place, Underground city",,,43.648429,-79.382280
198,M4Y,Downtown Toronto,Church and Wellesley,,,43.665860,-79.383160
199,M4Y,Downtown Toronto,Church and Wellesley,,,43.665860,-79.383160
200,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",,,43.662744,-79.321558


In [45]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto,ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ',address,' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  Toronto,ON  are 43.6534817, -79.3839347.


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  import sys


In [53]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

map_clusters